In [ ]:
!pip install torchvision

In [ ]:
import numpy as np

In [ ]:
from torchvision.datasets import EMNIST
from torchvision.transforms import PILToTensor,ToTensor

In [ ]:
training_data=EMNIST(root="data",split="digits",train=True,download=True,transform=ToTensor())
test_data=EMNIST(root="data",split="digits",train=False,download=True,transform=ToTensor())

In [ ]:
LEARNING_RATE=0.0001
EPOCHS=100000

In [ ]:
class Layer():
  def forward():
    pass

  def back_propogate():
    pass

In [ ]:
class DenseLayer(Layer):
  def __init__(self,input_size,output_size):
    self.weight = np.random.rand(input_size,output_size)
    self.bias = np.random.rand(1,output_size)
    # print(self.weight)
    # print(self.bias)
  def forward(self,input):
    self.input = input
    return input @ self.weight + self.bias

  def back_propogate(self,out_grad,learning_rate=0.1):
    bias_grad = np.sum(out_grad,axis=0) / out_grad.shape[0]
    weight_grad = self.input.T @ out_grad / out_grad.shape[0]
    input_grad = out_grad @ self.weight.T
    # print(f"out_grad shape: {out_grad.shape}")
    # print(f"input shape:{self.input.shape}")
    # print(f"weight_grad shape: {weight_grad.shape}")
    self.weight-=learning_rate * weight_grad
    self.bias-=learning_rate * bias_grad
    return input_grad

In [ ]:
class ReLU(Layer):
  def forward(self,input):
    self.input = input
    return np.where(input < 0 ,0,input)

  def back_propogate(self,out_grad,learning_rate=0.1):
    o= out_grad * np.where(self.input<=0,0,1)
    # print(f"in grad at relu = {o}")
    return o

In [ ]:
def mse(y,y_true):
  return np.sum(np.sum((y-y_true)**2,axis=1)) / y.shape[0]

In [ ]:
def mse_prime(y,y_true):
  return 2 * (y-y_true)

In [ ]:
def softmax(x):
  # Subtract the maximum value for numerical stability
  max_val = np.max(x, axis=1, keepdims=True)
  exp_x = np.exp(x - max_val)

  # Calculate softmax probabilities
  softmax_probs = exp_x / np.sum(exp_x, axis=1, keepdims=True)

  return softmax_probs


In [ ]:
def convert_to_one_hot(y_true, num_classes):
  # Ensure y_true is a 1D array
  y_true = np.squeeze(y_true)

  # Create an identity matrix of size num_classes
  identity_matrix = np.eye(num_classes)

  # Use y_true as indices to get the one-hot encoded matrix
  y_one_hot = identity_matrix[y_true]

  return y_one_hot

In [ ]:
def cross_entropy_loss(y,y_true):
  epsilon = 1e-15
  y=softmax(y)
  num_classes = y.shape[1]
  y_true = convert_to_one_hot(y_true,num_classes)
  loss = -1 * np.log(y+epsilon) * y_true
  return np.sum(np.sum(loss,axis=1),axis=0) / y.shape[0]

In [ ]:
def cross_entropy_loss_prime(y,y_true):
  s = softmax(y)
  num_classes = y.shape[1]
  y_true = convert_to_one_hot(y_true,num_classes)
  return s - y_true

In [ ]:
# output must not be softmaxed
def get_classification_accuracy(output,y_true):
  output = softmax(output)
  correct = 0
  for i in range(output.shape[0]):
    k = np.argmax(output[i])
    if k == y_true[i]:
      correct+=1
  return correct / output.shape[0]

In [ ]:
def train(x,y_true,loss_func,loss_prime,network:list[Layer]):

  for epoch in range(EPOCHS):
    output = x
    for layer in network:
      output = layer.forward(output)
    loss = loss_func(output,y_true)
    accuracy = get_classification_accuracy(output,y_true)
    print(f"Epoch {epoch} Loss: {loss} Accuracy: {accuracy*100} %")
    out_grad = loss_prime(output,y_true)
    l = len(network)
    for i in range(l-1,-1,-1):
      out_grad = network[i].back_propogate(out_grad,LEARNING_RATE)



Training

In [ ]:
x = np.array([t[0].squeeze().numpy().flatten() for t in training_data])
x = x[:1000]

In [ ]:
x.shape

(1000, 784)

In [ ]:
y_true = np.array([t[1] for t in training_data]).reshape((-1,1))
y_true = y_true[:1000]

In [ ]:
y_true.shape

(1000, 1)

In [ ]:
network:list[Layer]= [
    DenseLayer(784,64),
    ReLU(),
    DenseLayer(64,10),
    ReLU(),
    DenseLayer(10,10)
]

In [ ]:
train(x,y_true,cross_entropy_loss,cross_entropy_loss_prime,network)

In [ ]:
def test(x,y_true,loss_func,network:list[Layer]):

    output = x
    for layer in network:
      output = layer.forward(output)
    loss = loss_func(output,y_true)
    accuracy = get_classification_accuracy(output,y_true)
    print(f"Loss: {loss} Accuracy: {accuracy*100} %")

In [ ]:
x = np.array([t[0].squeeze().numpy().flatten() for t in test_data])
y_true = np.array([t[1] for t in test_data]).reshape((-1,1))

In [ ]:
print(x.shape)
print(y_true.shape)

(40000, 784)
(40000,)


In [ ]:
test(x,y_true,cross_entropy_loss,network)

Loss: 0.770522241258424 Accuracy: 76.25999999999999 %
